In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
# variance inflation factor
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import LinearRegression
# import traintestsplit
from sklearn.model_selection import train_test_split
# import crossval score
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.linear_model import LassoCV

# import metrics
from sklearn.metrics import mean_squared_error, r2_score
# set pd options to display all columns
pd.set_option('display.max_columns', None)

# Importing Aimes Housing Dataset


<!-- https://www.openintro.org/data/index.php?data=ames -->


In [ ]:
data = pd.read_csv('../data/aimes.csv')